# 6. Final Model Comparison and Conclusions

**Student:** Philipe Souza

## Purpose
- Collect results from all models (baseline, tuned, scratch CNN)
- Create comprehensive comparison visualizations
- Analyze relative performance across models
- Discuss transfer learning benefits
- Write conclusions and future recommendations

In [ ]:
# TODO: Load all model results
# - Baseline pretrained model metrics
# - Tuned variant (best configuration) metrics
# - Scratch CNN metrics

In [ ]:
# TODO: Create comparison table
# - Model name
# - Test accuracy
# - Precision, recall, F1-score
# - Training time (if available)
# - Key hyperparameters

In [ ]:
# TODO: Plot training curves for all models
# - All models on same training loss plot
# - All models on same validation loss plot
# - All models on same accuracy plot

In [ ]:
# TODO: Accuracy comparison bar plot
# - Compare test accuracy across all models

In [ ]:
# TODO: Baseline vs Tuned Variant comparison
# - Write 1-2 paragraphs analyzing:
# - What did tuning achieve?
# - How much improvement was there?
# - Was the improvement significant?
# - What does this tell us about the hyperparameter?

In [ ]:
# TODO: Pretrained vs Scratch CNN comparison
# - Write 1-2 paragraphs analyzing:
# - How much better is transfer learning?
# - Why does pretrained model perform better?
# - Trade-offs: performance vs training time
# - When would scratch CNN be preferable?

In [ ]:
# TODO: Overall best model selection
# - Which model performed best and why?
# - What factors contributed to its success?

In [ ]:
# TODO: Key findings summary (1-2 paragraphs)
# - Main insights about this dataset
# - Which approaches worked best
# - Key learnings about transfer learning vs custom architectures

In [ ]:
# TODO: Recommendations for future work
# - Other models to try
# - Additional hyperparameters to tune
# - Data augmentation improvements
# - Robustness testing ideas
# - Potential extensions (bonus work)

In [ ]:
# TODO: Limitations and considerations
# - Dataset limitations
# - Computational constraints
# - Validation methodology